
    
    
## <H2 align="center"> <font color='white'> **MAF1731B**: Microstructure and Trading Systems </H2>

### <H3 align="center"> <font color='white'> **Laboratory 4**: Market Microstructure </H3>

 <H5 align="center"> <em> Professor: Villalobos Ramos Omar Antonio <em>

 <p align="center"> <img style=" margin: 30px 15px 15px 15px;" src="https://pngimage.net/wp-content/uploads/2018/06/logo-iteso-png-5.png" width="350" height="150" /> 

 <strong> <H5 align="left"> Student:
Esteban Márquez Delgado <strong>

 *Registry*:
if700637: https://github.com/EstebanMqz/MyST_LAB_4

 <center> <font color= #555555> <font size = 4> November 2022 </a> <font color= #555555> <font size = 4> </a></font>

<hr style="border:0.02in solid gray"> </hr>

<center> <font color= #555555> <font size = 6> Laboratory 4 </font> <br> <br> <font color= #555555> <font size = 5> Market Microstructure  </font>

<hr style="border:0.02in solid gray"> </hr>

### <font color= #555555> <font size = 4> Abstract </font>

This document was prepared as delivery for Laboratory 4 of Microstructure and Trading Systems - MAF1731B class during the Autumn 2022 course at ITESO for the Bachelor of Financial Engineering. In order to analyze the market microstructure in criptocurrencies markets containing the following 3 sections:

1. **Consume CCXT data**: Download Criptocurrencies Order Books from exchanges (kraken, ftx, currencycom, coinmate, etc).

2. **Microstructure Visualization**: Build Time Series using the Order Books obtained in the previous point from the following elements of the Market Microstructure for each asset and exchange (Levels, Bid Volume, Ask Volume, Total Volume, Mid Price, VWAP.)

3. **Microstructure Modeling**: Estimation of Roll Effective Spread using the lagged series at t-5.



<font color= lightblue> <font size = 4> 0. Libraries and Dependencies: </font>

<font color= gray>

**Note**: To run this notebook it is necessary to have the following libraries contained in the [requirements.txt](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/requirements.txt) script of this project.

To install the libraries in the script run the following cell:

In [252]:
!pip install -r requirements.txt


ERROR: Invalid requirement: '"""' (from line 1 of requirements.txt)


If you prefer, manual installation can be done with the individual installation of the following libraries:

<font color= lightblue> <font size = 1> 
+ pandas >= 1.3.4
+ numpy >= 1.19.1
+ jupyter >= 1.0.0
+ chart_studio >= 1.1
+ plotly >= 4.14
+ logging >= 3.11
+ datetime >= 4.7
+ pandas_datareader >= 0.10.0
+ tk >= 0.10
+ random >= 3.11
+ plotly >= 5.60
+ yahoofinancials >= 1.60
+ scipy >= 1.7.3
+ regex >= 2022.3.15
+ matplotlib >= 3.5.1
+ fire >= 0.4.0
+ ccxt >= 2.1.33
+ time >= 3.11 


### <font color= lightblue> <font size = 4> 0.1 Introduction: </font>
In this laboratory we will use information from the Cryptocurrency Order Book of various international exchanges, with the primary objective of visualizing and modeling the microstructure of the market through the behavior of the Order Book and its characteristics such as; Matching Engine, Levels, Ask-Bid Volumes, etc.
<font color= gray>




##### *General help:*
<font color= lightgreen> **Note**: Progress checks titles in sections highlighted. <font color= white>

In order, to retrieve specific help from the scripts tree in the project execute the help function on the script or in specific modules from the following:
+ [data.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/data.py)
+ [functions.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/functions.py)
+ [visualizations.py](https://github.com/EstebanMqz/MyST_LAB_4/blob/main/visualizations.py)

In [2]:
## Libraries
import pandas as pd # Analysis and data management
import pandas_datareader as pdr
import pandas_datareader.data as web #Data extraction from the web
import numpy as np
from datetime import datetime, timedelta, date
from tkinter.ttk import Style
from tkinter import Y
import random as rand
import plotly.graph_objects as go #Plotly
import plotly.express as px
import warnings
from yahoofinancials import YahooFinancials # Yfinance 
import warnings
import yfinance as yf
import ccxt #Criptocurrencies
import logging as log #Exchanges 
from scipy.optimize import minimize # Optimization 
from collections import Counter #Counter
import time
import re
import io 
import glob
import os
#.Py archives in repo
import functions as fn 
import visualizations as vs
import data as dt
from os import path
import fire
%matplotlib inline

<font color= orange> <font size = 5> 1. Consume CCXT data: </font>

In order to know the exchanges avaliable in CCTX module in Python for Criptocurrencies data download, the following getattr function *find_exchanges* is executed:

In [3]:
help(fn.find_exchanges)

Help on function find_exchanges in module functions:

find_exchanges(features=None, is_authenticated=False)
    Function that returns avaliable cryptocurrencies exchanges in Python for CCTX module.
    
        Parameters:
        ----------
        features: None 
        is_authenticated: None 
    
        Returns:
        -------
        exchange_names: Array of authenticated exchanges avaliable in CCTX.



From all the 120 exchanges avaliable we pick **binance**, **ftx** and **bytetrade** because of their fast fetching.

In [4]:
#Available exchanges in CCTX
fn.find_exchanges()[:] #len 120

['aax',
 'alpaca',
 'ascendex',
 'bequant',
 'bibox',
 'bigone',
 'binance',
 'binancecoinm',
 'binanceus',
 'binanceusdm',
 'bit2c',
 'bitbank',
 'bitbay',
 'bitbns',
 'bitcoincom',
 'bitfinex',
 'bitfinex2',
 'bitflyer',
 'bitforex',
 'bitget',
 'bithumb',
 'bitmart',
 'bitmex',
 'bitopro',
 'bitpanda',
 'bitrue',
 'bitso',
 'bitstamp',
 'bitstamp1',
 'bittrex',
 'bitvavo',
 'bkex',
 'bl3p',
 'blockchaincom',
 'btcalpha',
 'btcbox',
 'btcex',
 'btcmarkets',
 'btctradeua',
 'btcturk',
 'buda',
 'bw',
 'bybit',
 'bytetrade',
 'cex',
 'coinbase',
 'coinbaseprime',
 'coinbasepro',
 'coincheck',
 'coinex',
 'coinfalcon',
 'coinmate',
 'coinone',
 'coinspot',
 'crex24',
 'cryptocom',
 'currencycom',
 'delta',
 'deribit',
 'digifinex',
 'exmo',
 'flowbtc',
 'fmfwio',
 'ftx',
 'ftxus',
 'gate',
 'gateio',
 'gemini',
 'hitbtc',
 'hitbtc3',
 'hollaex',
 'huobi',
 'huobijp',
 'huobipro',
 'idex',
 'independentreserve',
 'indodax',
 'itbit',
 'kraken',
 'kucoin',
 'kucoinfutures',
 'kuna',
 'lat

Now that we have selected the exchanges, we pick random criptocurrencies and verify that they are all avaliable in their platform.

In [5]:
exchanges= ['binance','ftx', 'bytetrade'] #Selected exchanges from list (can't modify because of fn and vs .py)
criptos = ['BTC/USDT','ETH/USDT','XRP/USDT'] #Criptos OBs (modify tickers here)

<font color= lightblue> <font size = 4> Order Book Levels & OHLCV for 3 exchanges and Criptos: </font>

Now we proceed to download Single Order Books from exchanges for criptocurrencies.

In [6]:
help(vs.cctx_download)

Help on function cctx_download in module visualizations:

cctx_download(lvls, cripto, exchange)
    Function that returns prices and quantities of given levels (as integer) Bids & Asks in an Orderbook
    for the specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and
    from the following exchanges: binance, ftx or ascendex (as string).
    
        Parameters:
        ----------
        lvls: Levels of bids/asks in the Order Book (int).
        cripto: Criptocurrency downloadable symbol (str).
        exchanges: Criptocurrency downloadable Exchange (str).
    
        Returns:
        -------
        levels_ob_bid: Prices and Quantities of Bids (pos [0]) and Asks (pos [1]) as dataframes.



We get n=25 levels for each OB, and proceed to download criptos and exchanges from their lists [0:2] (9).

We will start with binance and BTC/USDT.

In [7]:
params= 25, criptos[0], exchanges[0]
params

(25, 'BTC/USDT', 'binance')

+ <font color= lightgray> <font size = 2> *Bids*: </font>

In [8]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,16731.58,0.00838
1,16731.42,0.05000
2,16730.94,0.00903
3,16730.93,0.00914
4,16730.88,0.04736


+ <font color= lightgray> <font size = 2> *Asks*: </font>

In [9]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,16732.53,0.09478
1,16732.63,0.07000
2,16732.67,0.00634
3,16732.74,0.07059
4,16733.00,0.00639


In [10]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-10T06:05:42.923Z,16658.64,16664.14,16627.35,16639.21,217.47823


We plot an **Order Book** for desired tickers and exchanges the visualization *OBLvls*.

In [11]:
help(vs.OBLvls_hist)

Help on function OBLvls_hist in module visualizations:

OBLvls_hist(lvls, cripto, exchange)
    Function that plots an horizontal histogram in plotly for CriptoCurrencies OB/
    
        Parameters
        ----------
        lvls: Levels of bids/asks in the Order Book (int).
        cripto: Criptocurrency downloadable symbol (str).
        exchanges: Criptocurrency downloadable Exchange (str).
    
        x: Quantity (col) of the given cripto (str) for given lvls (int) in Order Book.
        y: Prices (col) of the given cripto (str) for given lvls (int) in Order Book.
    
        Returns
        -------
        Histogram of Order Book prices and quantities for n given lvls.



In [12]:
vs.OBLvls_hist(params[0],params[1], params[2])

We will proceed for binance and ETH/USDT.

In [13]:
params= 25, criptos[1], exchanges[0]
params

(25, 'ETH/USDT', 'binance')

In [14]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,1187.53,0.1680
1,1187.52,0.0125
2,1187.48,0.0421
3,1187.46,2.1720
4,1187.42,1.5039


In [15]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,1187.54,7.7553
1,1187.56,2.2757
2,1187.63,4.3436
3,1187.69,0.0500
4,1187.74,1.3433


In [16]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-10T06:05:46.961Z,1185.17,1185.94,1184.41,1185.16,434.1131


In [17]:
vs.OBLvls_hist(params[0],params[1], params[2])

We continue with the exchange binance and XRP/USDT.

In [18]:
params= 25, criptos[2], exchanges[0]
params

(25, 'XRP/USDT', 'binance')

In [19]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])
ob_lvls[0].head()

,price,quantity
Bid_Lvl,,
0,0.3709,11342.0
1,0.3708,28965.0
2,0.3707,17936.0
3,0.3706,31045.0
4,0.3705,19789.0


In [20]:
ob_lvls[1].head()

,price,quantity
Ask_Lvl,,
0,0.3710,41065.0
1,0.3711,26531.0
2,0.3712,20367.0
3,0.3713,18447.0
4,0.3714,30444.0


In [21]:
ob_lvls[2]

,exchange,timestamp,open,high,low,close,volume
OHLCV,,,,,,,
0,binance,2022-11-10T06:05:50.375Z,0.3691,0.3701,0.3686,0.3692,837143.0


In [22]:
vs.OBLvls_hist(params[0],params[1], params[2])

We could retrieve criptocurrencies with any of the 3 exchanges declared in our list *exchanges* but **to avoid being repetitive we will make one more random example** and we will conclude with their dataframes in progress check. For section section **2.** (*Microstructure Visualization*) a dataframe with different columns and plots that allow us to compare differences between exchanges will be created.

In [23]:
params= 25, criptos[rand.randint(0,2)], exchanges[rand.randint(0,2)]
params

(25, 'BTC/USDT', 'bytetrade')

In [24]:
ob_lvls=vs.cctx_download(params[0],params[1], params[2])

In [25]:
vs.OBLvls_hist(params[0],params[1], params[2])

In [26]:
v_params0= 25, criptos[0], exchanges[0]
v_params1= 25, criptos[0], exchanges[1]
v_params2= 25, criptos[0], exchanges[2]
v_params0, v_params1, v_params2

((25, 'BTC/USDT', 'binance'),
 (25, 'BTC/USDT', 'ftx'),
 (25, 'BTC/USDT', 'bytetrade'))

<strong> 
<font color= lightgreen> <font size = 3> Progress check 1 </font>

<font color= orange> <font size = 1> n = data fetch per exchange </font>

In [27]:
n=20 #Iterations 

In [28]:
help(vs.verif_ex1)

Help on function verif_ex1 in module visualizations:

verif_ex1(lvls, cripto, exchange, n)
    Function that returns Verif 1 for i levels (as integer) n times 
    from specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and the 
    following exchanges: binance, ftx or bytetrade (as string).
    
    Parameters:
    ----------
    lvls: Levels of bids/asks in the Order Book (int).
    cripto: Criptocurrency downloadable symbol (str).
    exchanges: Criptocurrency downloadable Exchange (str).
    n: Data retrieval per exchange (int).
    
    Returns:
    -------
    df1 = Section 1 Verification.
    df2 = OHLCV.



In [29]:
verif_0_B = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1_B = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2_B = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

Some differences between criptocurrencies within exchanges can be appreciated. Now we can realize which exchange from the sample can provide us with the lowest spread and best liquidity, and it is binance in this case. 

Clearly, prices are the most important factor for investments and that's why Mid-Price is going to be analyzed in the plots of section *2° (Microstructure Visualization)*. 

<font color= lightblue> <font size = 2> Verification BTC </font>

As it can be appreciated lower close prices of exchanges can be associated with bigger bid volumes mainly.

In [30]:
verif_df1 = fn.concatenate(verif_0_B[0], verif_1_B[0], verif_2_B[0], n)
verif_df1

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-10T06:05:59.871,16728.60,16728.42,1.00429,2.683860,0.18,16729.02
1,binance,2022-11-10T06:06:02.317,16730.80,16729.25,1.60794,4.760610,1.55,16732.85
2,binance,2022-11-10T06:06:05.040,16734.32,16732.71,0.97739,3.548600,1.61,16733.64
3,binance,2022-11-10T06:06:07.516,16727.99,16726.21,1.59091,23.874280,1.78,16731.62
4,binance,2022-11-10T06:06:09.998,16730.93,16729.24,1.96328,2.002590,1.69,16728.71
5,binance,2022-11-10T06:06:12.804,16731.85,16731.65,1.48109,1.594570,0.20,16731.33
6,binance,2022-11-10T06:06:15.445,16727.10,16726.63,1.01030,17.223460,0.47,16727.13
7,binance,2022-11-10T06:06:17.897,16727.46,16726.93,0.44989,17.062390,0.53,16726.48
8,binance,2022-11-10T06:06:20.871,16725.71,16725.18,2.28311,16.466700,0.53,16725.08
9,binance,2022-11-10T06:06:23.537,16727.23,16725.64,0.62740,19.231930,1.59,16727.82


<font color= lightblue> <font size = 2> OHLCV BTC </font>

For each timeframe a candle is formed and it illustrates how much Volatility every timestamp shows within exchanges. This is important to know for algorithmic trading, not to mention the fact that stock exchanges do not provide candle data in *ms* for many reasons.

In [31]:
verif_df1_OH = fn.concatenate(verif_0_B[1], verif_1_B[1], verif_2_B[1], n)
verif_df1_OH

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-10T06:05:59.871,16781.55,16788.88,16723.12,16729.02,353.35313
1,binance,2022-11-10T06:06:02.317,16728.43,16736.13,16728.42,16732.85,18.39390
2,binance,2022-11-10T06:06:05.040,16728.43,16736.13,16726.01,16733.64,36.84977
3,binance,2022-11-10T06:06:07.516,16728.43,16738.73,16726.01,16731.62,43.96410
4,binance,2022-11-10T06:06:09.998,16728.43,16738.73,16726.01,16728.71,51.92495
5,binance,2022-11-10T06:06:12.804,16728.43,16738.73,16726.01,16731.33,58.41361
6,binance,2022-11-10T06:06:15.445,16728.43,16738.73,16724.00,16727.13,72.97992
7,binance,2022-11-10T06:06:17.897,16728.43,16738.73,16724.00,16726.48,75.66190
8,binance,2022-11-10T06:06:20.871,16728.43,16738.73,16724.00,16725.08,79.16868
9,binance,2022-11-10T06:06:23.537,16728.43,16738.73,16724.00,16727.82,87.25839


<font color= lightblue> <font size = 2> Verification ETH </font>

In [32]:
v_params0= 25, criptos[1], exchanges[0]
v_params1= 25, criptos[1], exchanges[1]
v_params2= 25, criptos[1], exchanges[2]
v_params0, v_params1, v_params2

((25, 'ETH/USDT', 'binance'),
 (25, 'ETH/USDT', 'ftx'),
 (25, 'ETH/USDT', 'bytetrade'))

In [33]:
verif_0_E = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1_E = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2_E = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

In [34]:
verif_df2 = fn.concatenate(verif_0_E[0], verif_1_E[0], verif_2_E[0], n)
verif_df2

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-10T06:09:03.590,1186.78,1186.77,133.8192,59.8967,0.01,1186.68
1,binance,2022-11-10T06:09:06.157,1186.46,1186.45,63.9908,62.4058,0.01,1186.46
2,binance,2022-11-10T06:09:08.823,1187.83,1187.82,67.8517,63.3495,0.01,1187.81
3,binance,2022-11-10T06:09:11.685,1187.42,1187.41,118.3054,67.4575,0.01,1187.41
4,binance,2022-11-10T06:09:14.705,1187.68,1187.67,73.6399,45.8913,0.01,1187.63
5,binance,2022-11-10T06:09:17.292,1187.98,1187.97,66.3074,60.9237,0.01,1187.97
6,binance,2022-11-10T06:09:20.154,1187.27,1187.26,54.4637,45.5810,0.01,1187.26
7,binance,2022-11-10T06:09:22.666,1187.37,1187.36,102.6880,53.5459,0.01,1187.36
8,binance,2022-11-10T06:09:25.323,1187.37,1187.36,74.1355,47.2142,0.01,1187.36
9,binance,2022-11-10T06:09:27.925,1187.85,1187.74,79.2262,78.6530,0.11,1187.85


<font color= lightblue> <font size = 2> OHLCV ETH </font>

In [35]:
verif_df2_OH = fn.concatenate(verif_0_E[1], verif_1_E[1], verif_2_E[1], n)
verif_df2_OH

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-10T06:09:03.590,1187.14,1187.68,1186.52,1186.68,320.8689
1,binance,2022-11-10T06:09:06.157,1187.14,1187.68,1186.45,1186.46,329.9121
2,binance,2022-11-10T06:09:08.823,1187.14,1187.81,1186.29,1187.81,440.6467
3,binance,2022-11-10T06:09:11.685,1187.14,1187.83,1186.29,1187.41,651.1713
4,binance,2022-11-10T06:09:14.705,1187.14,1187.83,1186.29,1187.63,657.9066
5,binance,2022-11-10T06:09:17.292,1187.14,1187.98,1186.29,1187.97,673.0728
6,binance,2022-11-10T06:09:20.154,1187.14,1187.98,1186.29,1187.26,690.4015
7,binance,2022-11-10T06:09:22.666,1187.14,1187.98,1186.29,1187.36,707.7800
8,binance,2022-11-10T06:09:25.323,1187.14,1187.98,1186.29,1187.36,720.7998
9,binance,2022-11-10T06:09:27.925,1187.14,1187.98,1186.29,1187.85,737.2562


<font color= lightblue> <font size = 2> Verification XRP </font>

In [36]:
v_params0= 25, criptos[2], exchanges[0]
v_params1= 25, criptos[2], exchanges[1]
v_params2= 25, criptos[2], exchanges[2]
v_params0, v_params1, v_params2

((25, 'XRP/USDT', 'binance'),
 (25, 'XRP/USDT', 'ftx'),
 (25, 'XRP/USDT', 'bytetrade'))

In [37]:
verif_0_X = vs.verif_ex1(v_params0[0],v_params0[1], v_params0[2], n)
verif_1_X = vs.verif_ex1(v_params1[0],v_params1[1], v_params1[2], n)
verif_2_X = vs.verif_ex1(v_params2[0],v_params2[1], v_params2[2], n)

In this example, bigger spreads can be detected on bytetrade for XRP. On the contrary, binance has lower spreads and bigger volumes as well.

In [38]:
verif_df3 = fn.concatenate(verif_0_X[0], verif_1_X[0], verif_2_X[0], n)
verif_df3

,exchange,timestamp,ask,bid,ask_volume,bid_volume,spread,close_price
0,binance,2022-11-10T06:12:11.307,0.373100,0.373000,6.053660e+05,8.244020e+05,0.000100,0.373100
1,binance,2022-11-10T06:12:14.216,0.373500,0.373400,5.816640e+05,6.751680e+05,0.000100,0.373400
2,binance,2022-11-10T06:12:16.695,0.373700,0.373600,1.467702e+06,6.652810e+05,0.000100,0.373700
3,binance,2022-11-10T06:12:19.186,0.373300,0.373200,6.703410e+05,5.820930e+05,0.000100,0.373300
4,binance,2022-11-10T06:12:21.860,0.373400,0.373300,6.819040e+05,6.444780e+05,0.000100,0.373300
5,binance,2022-11-10T06:12:24.440,0.373300,0.373200,7.095710e+05,8.879170e+05,0.000100,0.373300
6,binance,2022-11-10T06:12:26.973,0.373300,0.373200,7.138300e+05,9.350910e+05,0.000100,0.373200
7,binance,2022-11-10T06:12:29.490,0.373300,0.373200,7.344290e+05,1.038295e+06,0.000100,0.373200
8,binance,2022-11-10T06:12:32.039,0.373000,0.372900,5.326530e+05,8.955100e+05,0.000100,0.372900
9,binance,2022-11-10T06:12:34.505,0.372900,0.372800,8.052100e+05,9.378020e+05,0.000100,0.372900


<font color= lightblue> <font size = 2> OHLCV XRP </font>

In [39]:
verif_df3_OH = fn.concatenate(verif_0_X[1], verif_1_X[1], verif_2_X[1], n)
verif_df3_OH

,exchange,timestamp,open,high,low,close,volume
0,binance,2022-11-10T06:12:11.307,0.372600,0.373100,0.372600,0.373100,18504.000000
1,binance,2022-11-10T06:12:14.216,0.372600,0.373400,0.372600,0.373400,72064.000000
2,binance,2022-11-10T06:12:16.695,0.372600,0.373700,0.372600,0.373700,84994.000000
3,binance,2022-11-10T06:12:19.186,0.372600,0.373700,0.372600,0.373300,175291.000000
4,binance,2022-11-10T06:12:21.860,0.372600,0.373700,0.372600,0.373300,187730.000000
5,binance,2022-11-10T06:12:24.440,0.372600,0.373700,0.372600,0.373300,205444.000000
6,binance,2022-11-10T06:12:26.973,0.372600,0.373700,0.372600,0.373200,205560.000000
7,binance,2022-11-10T06:12:29.490,0.372600,0.373700,0.372600,0.373200,211031.000000
8,binance,2022-11-10T06:12:32.039,0.372600,0.373700,0.372600,0.372900,412926.000000
9,binance,2022-11-10T06:12:34.505,0.372600,0.373700,0.372600,0.372900,423694.000000


<font color= orange> <font size = 5> 2. Microstructure Visualization </font>

Now we are going to obtain total_volumes, mid_prices and VWAP metric for criptocurrencies in every exchange.

In [40]:
help(vs.Micro)

Help on function Micro in module visualizations:

Micro(lvls, cripto, exchange, n, ts)
    Function that returns Microstructure OB and OHLCV for i levels (as integer) n times 
    from specified criptocurrency ('BTC/USDT','ETH/USDT','XRP/USDT', or others) and the 
    following exchanges: binance, ftx or bytetrade (as string).
    
    Parameters:
    ----------
    lvls: Levels of bids/asks in the Order Book (int).
    cripto: Criptocurrency downloadable symbol (str).
    exchanges: Criptocurrency downloadable Exchange (str).
    n: Data retrieval per exchange (int).
    ts: timesleep (s) required in between data retrieval.
    
    Returns:
    -------
    Micro[0]: Order Books Data.
    Micro[1]: OHLCV Data.



<strong> 
<font color= lightgreen> <font size = 3> Progress check 2  </font> <strong>

Dataframes

<font color= lightblue> <font size = 2> Verification BTC </font>

In [41]:
params0= 25, criptos[0], exchanges[0]
params1= 25, criptos[0], exchanges[1]
params2= 25, criptos[0], exchanges[2]
params0,params1,params2

((25, 'BTC/USDT', 'binance'),
 (25, 'BTC/USDT', 'ftx'),
 (25, 'BTC/USDT', 'bytetrade'))

In [42]:
MS_0_B = vs.Micro(params0[0],params0[1], params0[2], n, .01)
MS_1_B = vs.Micro(params1[0],params1[1], params1[2], n, .01)
MS_2_B = vs.Micro(params2[0],params2[1], params2[2], n, .01)

Bid and Asks Volumes in the Microstructure determine the demand and it's a factor for price movements. This is also true for the VWAP which is a trend detector by reflecting the price adjusted by its volume.

In [43]:
MS_B = fn.concatenate(MS_0_B[0], MS_1_B[0], MS_2_B[0], n)
MS_B

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2022-11-10T06:15:12.709,25,0.65634,1.030990,1.687330,16669.445,26548.354906
1,binance,2022-11-10T06:15:14.111,25,0.79225,2.178790,2.971040,16669.970,22280.484608
2,binance,2022-11-10T06:15:15.791,25,1.91793,3.751170,5.669100,16664.355,19602.893535
3,binance,2022-11-10T06:15:17.605,25,0.72454,3.629350,4.353890,16664.510,20491.924352
4,binance,2022-11-10T06:15:19.634,25,1.15110,2.064200,3.215300,16655.560,21835.152209
5,binance,2022-11-10T06:15:21.524,25,1.44438,2.146730,3.591110,16655.730,21293.479232
6,binance,2022-11-10T06:15:23.146,25,0.55711,0.837820,1.394930,16662.220,28607.051786
7,binance,2022-11-10T06:15:24.935,25,0.49497,2.200090,2.695060,16661.950,22844.284918
8,binance,2022-11-10T06:15:26.463,25,1.23673,4.275740,5.512470,16653.250,19673.944922
9,binance,2022-11-10T06:15:28.054,25,1.42952,4.355440,5.784960,16653.560,19531.956180


<font color= lightblue> <font size = 2> Verification ETH </font>

In [44]:
params0= 25, criptos[1], exchanges[0]
params1= 25, criptos[1], exchanges[1]
params2= 25, criptos[1], exchanges[2]
params0,params1,params2

((25, 'ETH/USDT', 'binance'),
 (25, 'ETH/USDT', 'ftx'),
 (25, 'ETH/USDT', 'bytetrade'))

In [45]:
MS_0_E = vs.Micro(params0[0],params0[1], params0[2], n, .01)
MS_1_E = vs.Micro(params1[0],params1[1], params1[2], n, .01)
MS_2_E = vs.Micro(params2[0],params2[1], params2[2], n, .01)

In [46]:
MS_E = fn.concatenate(MS_0_E[0], MS_1_E[0], MS_2_E[0], n)
MS_E

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2022-11-10T06:17:10.539,25,48.4213,62.5787,111.0000,1180.285,1190.913243
1,binance,2022-11-10T06:17:12.067,25,73.9463,85.8430,159.7893,1180.465,1187.847666
2,binance,2022-11-10T06:17:13.847,25,40.5972,67.7131,108.3103,1180.385,1191.278225
3,binance,2022-11-10T06:17:15.455,25,50.4541,70.3612,120.8153,1180.195,1189.958630
4,binance,2022-11-10T06:17:17.408,25,54.9993,91.4939,146.4932,1180.510,1188.548599
5,binance,2022-11-10T06:17:19.597,25,60.1144,70.1004,130.2148,1180.505,1189.565867
6,binance,2022-11-10T06:17:21.202,25,74.3735,84.2988,158.6723,1180.855,1188.292131
7,binance,2022-11-10T06:17:23.345,25,80.3936,83.3824,163.7760,1180.955,1188.160825
8,binance,2022-11-10T06:17:25.079,25,103.3962,93.2204,196.6166,1180.805,1186.805648
9,binance,2022-11-10T06:17:26.619,25,128.3629,113.5354,241.8983,1180.765,1185.641266


<font color= lightblue> <font size = 2> Verification XRP </font>

In [47]:
params0= 25, criptos[2], exchanges[0]
params1= 25, criptos[2], exchanges[1]
params2= 25, criptos[2], exchanges[2]
params0,params1,params2

((25, 'XRP/USDT', 'binance'),
 (25, 'XRP/USDT', 'ftx'),
 (25, 'XRP/USDT', 'bytetrade'))

In [48]:
MS_0_X = vs.Micro(params0[0],params0[1], params0[2], n, .01)
MS_1_X = vs.Micro(params1[0],params1[1], params1[2], n, .01)
MS_2_X = vs.Micro(params2[0],params2[1], params2[2], n, .01)

In [49]:
MS_X = fn.concatenate(MS_0_X[0], MS_1_X[0], MS_2_X[0], n)
MS_X

,exchange,timestamp,level,ask_volume,bid_volume,total_volume,mid_price,vwap
0,binance,2022-11-10T06:19:15.235,25,659197.000000,2.026060e+06,2.685257e+06,0.368950,0.368900
1,binance,2022-11-10T06:19:16.795,25,677497.000000,1.904598e+06,2.582095e+06,0.368650,0.368600
2,binance,2022-11-10T06:19:18.749,25,639580.000000,1.976379e+06,2.615959e+06,0.368750,0.368700
3,binance,2022-11-10T06:19:20.899,25,629065.000000,2.042268e+06,2.671333e+06,0.368950,0.368900
4,binance,2022-11-10T06:19:22.838,25,485307.000000,2.016367e+06,2.501674e+06,0.369050,0.369000
5,binance,2022-11-10T06:19:24.681,25,517734.000000,2.010432e+06,2.528166e+06,0.369050,0.369000
6,binance,2022-11-10T06:19:26.415,25,533270.000000,2.012094e+06,2.545364e+06,0.369050,0.369000
7,binance,2022-11-10T06:19:27.958,25,529229.000000,2.026291e+06,2.555520e+06,0.369050,0.369000
8,binance,2022-11-10T06:19:30.006,25,574062.000000,2.061518e+06,2.635580e+06,0.368850,0.368800
9,binance,2022-11-10T06:19:31.750,25,565265.000000,2.057429e+06,2.622694e+06,0.368750,0.368700


<strong> 
<font color= lightgreen> <font size = 3> Progress check 2  </font> <strong>

Visualization

In [50]:
help(vs.Plot_line)

Help on function Plot_line in module visualizations:

Plot_line(data_ms, cripto, title)
    Function that plots Mid-Prices (y) and timestamps (x) for exchanges in data.
    Parameters:
    ----------
    data_ms: Microstructure data for column extraction ['exchanges', 'Mid_Prices'] and index timestamp as datetime.
    cripto: Downloadable cripto symbol.
    title: Title of facet_cols plotly lines before criptocurrency symbol (ex: 'Volume of': XRP/USDT)
    
    Returns:
    -------
    facet_col plots for Mid-Prices (col) during timestamps of data (index) for exchanges (col).



In [51]:
vs.Plot_line(MS_B, criptos[0], "Mid-Price of")

In [52]:
vs.Plot_line(MS_E, criptos[1], "Mid-Price of")

In [53]:
vs.Plot_line(MS_X, criptos[2], "Mid-Price of")

As it can be appreciated, Mid-Prices have different values in every exchange. Given this fact, there are entities that aim to reduce the spread between exchanges and they intervene creating virtual demand but let's not ignore that this is also traduced into gains for them also known as market makers.

<font color= orange> <font size = 5> 3. Microstructure Modelling: </font>

<strong> 
<font color= lightgreen> <font size = 3> Progress check 3  </font> <strong>

<font color= lightblue> <font size = 2> BTC Spreads </font>

In [280]:
Roll_B=roll_model(verif_df1, 5)
Roll_B.reset_index(drop=True)

,exchange,timestamp,close_price,spread,effective spread
0,binance,2022-11-10T06:06:12.804,16731.33,0.20,0.760614
1,binance,2022-11-10T06:06:15.445,16727.13,0.47,5.093709
2,binance,2022-11-10T06:06:17.897,16726.48,0.53,4.735624
3,binance,2022-11-10T06:06:20.871,16725.08,0.53,1.139883
4,binance,2022-11-10T06:06:23.537,16727.82,1.59,3.140679
5,binance,2022-11-10T06:06:26.200,16728.08,1.00,3.248384
6,binance,2022-11-10T06:06:29.400,16725.75,0.74,2.481048
7,binance,2022-11-10T06:06:31.892,16732.71,1.03,4.286319
8,binance,2022-11-10T06:06:34.739,16725.89,1.48,6.277399
9,binance,2022-11-10T06:06:37.440,16727.37,1.11,6.727397


<font color= lightblue> <font size = 2> ETH Spreads </font>

In [254]:
roll_model(verif_df2, 5)


,exchange,timestamp,spread,close_price,effective spread
5,binance,2022-11-10T06:09:17.292,0.01,1187.97,1.269554
6,binance,2022-11-10T06:09:20.154,0.01,1187.26,1.449816
7,binance,2022-11-10T06:09:22.666,0.01,1187.36,0.884666
8,binance,2022-11-10T06:09:25.323,0.01,1187.36,0.277308
9,binance,2022-11-10T06:09:27.925,0.11,1187.85,1.075128
10,binance,2022-11-10T06:09:30.648,0.01,1187.82,1.024109
11,binance,2022-11-10T06:09:33.214,0.01,1187.52,0.168127
12,binance,2022-11-10T06:09:35.773,0.01,1187.26,0.697615
13,binance,2022-11-10T06:09:38.491,0.01,1187.38,0.861665
14,binance,2022-11-10T06:09:41.361,0.01,1186.97,0.566304


<font color= lightblue> <font size = 2> XRP Spreads </font>

In [260]:
roll_model(verif_df3, 5)


,exchange,timestamp,spread,close_price,effective spread
5,binance,2022-11-10T06:12:24.440,0.000100,0.373300,0.000183
6,binance,2022-11-10T06:12:26.973,0.000100,0.373200,0.000574
7,binance,2022-11-10T06:12:29.490,0.000100,0.373200,0.000420
8,binance,2022-11-10T06:12:32.039,0.000100,0.372900,0.000058
9,binance,2022-11-10T06:12:34.505,0.000100,0.372900,0.000200
10,binance,2022-11-10T06:12:37.074,0.000100,0.373200,0.000231
11,binance,2022-11-10T06:12:39.724,0.000100,0.373700,0.000400
12,binance,2022-11-10T06:12:42.605,0.000100,0.373300,0.000321
13,binance,2022-11-10T06:12:45.459,0.000100,0.373000,0.000383
14,binance,2022-11-10T06:12:48.151,0.000100,0.372800,0.000379


### <font color= orange> <font size = 5> 4. Conclusions: </font>

<font color= lightgray>

### <font color= orange> <font size = 5> 5. Bibliography: </font>

<font color= lightgray>

+ Villalobos, O, 2021. **Python Project Template**. Available at: https://github.com/OmarVillalobos/python-project. 
(Accessed: November 5, 2022). 

+ Financial Engineering, **Marketmicrostructure.pptx**. ITESO -  Microstructure and Trading Systems (ITE1731 - DMAF). 
Available at: https://docs.google.com/presentation/d/1uYyMERKKGeZ0MPqbJ3bcir9lfpzbeAH9ZMmxbYysokQ/edit?usp=sharing (Accessed: November 5, 2022). 

+ Muñoz E., J **Roll Model** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/Roll%20.pdf (Accessed: November 5, 2022).

+ Muñoz E., J **Asset Pricing Theory** *(spanish)*. ITESO - (ITE1731 - DMAF) Available at: https://github.com/EstebanMqz/MyST_LAB_4/blob/main/APT.pdf (Accessed: November 5, 2022). 